In [1]:
'''
Test - 2.11  Internal Trigger Input test
Equipment required -

1) Lucid device - LSX4091D
2) Oscilloscope	Configure the scope as follows:
a.	Amplitude: 500 mV per division
b.	Time base: 50% of the LSX4091 Trigger Timer
c.	Trigger mode: Triggered
d.	Trigger Level: 0.5


 '''
import numpy as np
import pyvisa as visa
import socket
import time
import re  # Set the IP Address and port

handle = 'TCPIP0::192.168.7.1::5025::SOCKET'


def send_scpi_cmd(cmd, handle):
    try:
        resourceManager = visa.ResourceManager()
        session = resourceManager.open_resource(handle)

        # Need to define the termination string
        session.write_termination = '\n'
        session.read_termination = '\n'

        session.write(cmd)
        session.close()
    except Exception as e:
        print('[!] Exception: ' + str(e))


def send_scpi_query(query, handle):
    try:
        resourceManager = visa.ResourceManager()
        session = resourceManager.open_resource(handle)

        # Need to define the termination string
        session.write_termination = '\n'
        session.read_termination = '\n'

        response = str(session.query(query))
        response =response.split(";")[0]
        session.close()
        return response

    except Exception as e:
        print('[!] Exception: ' + str(e))


send_scpi_cmd('*CLS', handle)
query = '*IDN?'
temp = send_scpi_query(query, handle)
print("Connected to :", temp)

Connected to : Tabor Electronics,LSX2091D,20240101,1.0.35


In [2]:
power_in_dBm=5
trigger_timer = [1e-3,1e-4]
for i in trigger_timer:
    send_scpi_cmd(':FREQuency 100e6', handle)
    send_scpi_cmd(':POWer {0}'.format(power_in_dBm), handle)
    freq_query = send_scpi_query(':FREQuency?', handle)
    power_query = send_scpi_query(':POWer?', handle)
    send_scpi_cmd(':OUTPut ON', handle)
    send_scpi_cmd(':INIT:CONT OFF',handle)
    send_scpi_cmd(':TRIG:SOUR INT',handle)
    send_scpi_cmd(':TRIG:SOUR TIM;',handle)
    send_scpi_cmd(':TRIG:EDG POS',handle)
    send_scpi_cmd(':TRIG:TIM:TIME {0}'.format(i),handle)

    # AM modulation setting
    send_scpi_cmd(':AM:SOUR INT',handle)
    send_scpi_cmd(':AM:INT:FREQ 100e3',handle)
    send_scpi_cmd(':AM:DEPT 50',handle)
    send_scpi_cmd(':AM ON',handle)
    print('Check for frequency ',freq_query)
    print('and Power ',power_in_dBm,'dBm')
    print('Press input for next parameter')
    input()
    send_scpi_cmd(':OUTPut OFF', handle)
print("test completed")


Check for frequency  100000000
and Power  5 dBm
Press input for next parameter

Check for frequency  100000000
and Power  5 dBm
Press input for next parameter

test completed
